# Discrimination Against the Handicapped

Do subjects systematically evaluate qualifications differently according to the candidate's handicap?  If so, which handicaps produce the same evaluations?

In [ ]:
# standard library imports
import warnings

warnings.simplefilter('ignore', category=FutureWarning)

# 3rd party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi

sns.set()

In [ ]:
df = pd.read_csv('case0601.csv')

# the Handicap column has None as items where there was no handicap.  Must
# replace that with a suitable string in order to get that category noticed.
df.loc[df['Handicap'].isnull(), 'Handicap'] = 'None'

## Robustness to Assumptions

In [ ]:
df.groupby('Handicap').describe()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[12, 6.4])
_ = sns.boxplot(data=df, x='Handicap', y='Score', ax=axes[0])
_ = sns.kdeplot(data=df, x='Score', hue='Handicap', ax=axes[1])

In [ ]:
sm.graphics.qqplot(df['Score'], fit=True, line='45')
plt.show()

There do not appear to be any violations of normality, equal variance, or independence.

# Are any handicap groups different?

$
\begin{align}
H_0: &\: \mu_{amputee} = \mu_{crutches} = \mu_{hearing} = \mu_{none} = \mu_{wheelchair} \\
H_a: &\: \text{at least one} \: \mu_i \ne \mu_j \: \text{for} \: i, j \in \{amputee, crutches, hearing, wheelchair\} \\
\end{align}
$

In [ ]:
model = smf.ols('Score ~ C(Handicap, Treatment(reference="None"))', data=df).fit()
model.summary()

There is only weak evidence that any of the perceived handicaps differ from the control of no handicap ("None").

In [ ]:
sm.stats.anova_lm(model)

There is moderate evidence that at least one group is different from the others, however ($F_{4,65}$ = 2.8616, $p$-value = 0.0301).

In [ ]:
mc = multi.MultiComparison(df['Score'], df['Handicap'])
res = mc.tukeyhsd(alpha=0.05)
print(res.summary())

As one might have guessed, the strongest evidence of a difference is between Crutches and Hearing ($p$-value = 0.0278)

# Calculate the comparison of Amputee/Hearing to Crutches/Wheelchair.

In [ ]:
model.params

In [ ]:
r = np.zeros_like(model.params)
r[1] = r[3] = 0.5
r[2] = r[4] = -0.5
model.t_test(r)

There is strong evidence of a difference between the average scores of the amputee and hearing groups as compared with the crutches and wheelchair groups ($t_{0.95, 65}$ = -3.192, $p$-value = 0.002).  We are 95% confident that the true difference is between 2.264 and 0.521 points higher for the crutches/wheelchair groups.